### Part A

In [1]:
# HERE YOU WILL WRITE CODE TO TEST A NUMBER OF PREDICTORS
# AND FINALLY CHOOSE AND TRAIN THE PREDICTOR THAT YOU WILL BE USING FOR PART B

In [2]:
# import the necessary ibraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import random
import warnings
from scipy import stats
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from matplotlib import pyplot
warnings.filterwarnings('ignore')

In [3]:
#load the dataset
dataset = pd.read_csv('CE802_P3_Data.csv')
dataset.shape
#(features, labels)

(1500, 37)

In [4]:
# dataset describe
dataset.describe()

,F1,F2,F3,F4,F6,F7,F8,F9,F10,F11,...,F27,F28,F29,F30,F31,F32,F33,F35,F36,Target
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,49.361787,-209.530120,-200.023327,107.093500,14.481180,-157.306927,11470.972200,-4402.414547,-169.629347,24706.125200,...,7.938667,-1065.452867,891.690960,-74.581840,-8.218040,11.900560,18.324180,-326.214980,-1191.986500,856.493547
std,41.155295,82.215588,41.185646,30.352973,12.475389,42.300767,9210.333946,845.754592,59.791412,8924.332099,...,3.658756,3018.544636,579.053729,12.983768,5.582196,4.332523,4.214935,90.079627,299.256174,1222.860406
min,-162.270000,-556.960000,-393.320000,11.790000,-39.600000,-412.790000,-21849.390000,-9473.680000,-365.080000,-6937.110000,...,0.000000,-11673.120000,-1106.120000,-145.470000,-35.020000,-10.350000,-0.090000,-637.170000,-2422.210000,-2685.920000
25%,29.140000,-251.310000,-220.295000,85.802500,7.770000,-178.617500,5268.525000,-4813.235000,-209.485000,18626.070000,...,6.000000,-3098.372500,513.460000,-80.400000,-10.590000,9.860000,16.220000,-386.137500,-1386.100000,17.235000
50%,49.650000,-209.010000,-199.635000,107.665000,14.160000,-157.685000,11274.165000,-4415.220000,-170.560000,24601.425000,...,8.000000,-1031.230000,871.410000,-74.280000,-6.900000,11.900000,18.350000,-325.320000,-1192.645000,624.735000
75%,70.155000,-168.325000,-177.860000,126.560000,21.030000,-136.702500,17816.257500,-3992.115000,-129.840000,30487.492500,...,10.000000,849.520000,1281.080000,-68.040000,-4.195000,13.980000,20.325000,-263.640000,-996.160000,1609.167500
max,243.560000,173.340000,-0.570000,219.010000,88.230000,69.400000,38994.540000,220.360000,47.440000,50405.070000,...,20.000000,10361.960000,3744.480000,-26.850000,-0.480000,31.520000,35.920000,-11.370000,-166.580000,3836.440000


In [5]:
# column informations
print(dataset.columns)

# frequency table
# showing the unique values in each columns
count_  = dataset.nunique().to_frame(name = 'Counts')
count_

Index(['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F35', 'F36', 'Target'],
      dtype='object')


,Counts
F1,1409
F2,1423
F3,1406
F4,1412
F5,5
F6,910
F7,1410
F8,1499
F9,1487
F10,1382


In [6]:
# Count null values from the dataset

null_values = dataset.isnull().sum().to_frame(name = 'NULL_COUNTS')
null_values = null_values[null_values['NULL_COUNTS']>0]
null_features = null_values.index
null_values

,NULL_COUNTS


In [7]:
# search for duplicates in the dataset 
dataset.duplicated().sum() # mo duplicate values found

0

In [8]:
# get the data types of the dataset
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   F1      1500 non-null   float64
 1   F2      1500 non-null   float64
 2   F3      1500 non-null   float64
 3   F4      1500 non-null   float64
 4   F5      1500 non-null   object 
 5   F6      1500 non-null   float64
 6   F7      1500 non-null   float64
 7   F8      1500 non-null   float64
 8   F9      1500 non-null   float64
 9   F10     1500 non-null   float64
 10  F11     1500 non-null   float64
 11  F12     1500 non-null   float64
 12  F13     1500 non-null   float64
 13  F14     1500 non-null   float64
 14  F15     1500 non-null   float64
 15  F16     1500 non-null   float64
 16  F17     1500 non-null   float64
 17  F18     1500 non-null   float64
 18  F19     1500 non-null   float64
 19  F20     1500 non-null   float64
 20  F21     1500 non-null   float64
 21  F22     1500 non-null   float64
 22  

In [9]:
X = dataset.iloc[:, 0:-1]
y =dataset.iloc[:, -1]
# For the categorical features, we will encode them with One-Hot encoding
categorical_pipeline = Pipeline(
    steps=[
        ("onehot-encoder", OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ]
)

# For the numeric features, I will choose StandardScaler so that the features have 0 mean and a variance of 1:
numeric_pipeline = Pipeline(
    steps=[("scale", StandardScaler())]
)

# categorical and numeric features
cat_cols = X.select_dtypes(exclude="number").columns
num_cols = X.select_dtypes(include="number").columns

full_processor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols),
        ("categorical", categorical_pipeline, cat_cols),
    ]
)
# Apply preprocessing
X_processed = full_processor.fit_transform(X)
scaler_y = StandardScaler()
y_processed = scaler_y.fit_transform(y.values.reshape(-1, 1))

X_train, X_valid, y_train, y_valid = train_test_split(
    X_processed, y_processed, random_state=100
)

In [15]:
np.savetxt("foo.csv", X_processed, delimiter=",")

In [10]:
# Define the models to be compared
regressors = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(objective='reg:squarederror'),
}

# Define the parameters for each model
param_grid = {
    'Linear Regression': {},
    'Ridge': {'alpha': [0.1, 1, 10, 15, 20]},
    'Decision Tree': {'max_depth': [3, 5, 7, 10, 20]},
    'Random Forest': {'n_estimators': [10, 50, 100, 150, 200, 300]},
    'XGBoost': {'max_depth': [3, 5, 7], 'n_estimators': [300, 400, 500, 600, 700, 750]},
}

rmse = []
mae = []
r2 = []
# Use GridSearchCV to compare the models
for name, model in regressors.items():
    reg = GridSearchCV(model, param_grid[name], cv=5, scoring=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'], refit='neg_mean_squared_error')
    reg.fit(X_train, y_train)
    rmse_ = np.sqrt(mean_squared_error(y_valid, reg.predict(X_valid)))
    rmse.append(rmse_)
    mae_ = mean_absolute_error(y_valid, reg.predict(X_valid))
    mae.append(mae_)
    r2_ = r2_score(y_valid, reg.predict(X_valid))
    r2.append(r2_)
    print(f'{name} model best score with RMSE: {rmse_}')
    print(f'{name} model best score with RMSE: {mae_}')
    print(f'{name} model best score with RMSE: {r2_}')
    print(f'{name} model best params: {reg.best_params_}')
    print('\n')

Linear Regression model best score with RMSE: 0.535299325645433
Linear Regression model best score with RMSE: 0.42624233132559736
Linear Regression model best score with RMSE: 0.6977472014953896
Linear Regression model best params: {}


Ridge model best score with RMSE: 0.5346820082303431
Ridge model best score with RMSE: 0.4244237651533437
Ridge model best score with RMSE: 0.6984439269352322
Ridge model best params: {'alpha': 15}


Decision Tree model best score with RMSE: 0.7373488677991545
Decision Tree model best score with RMSE: 0.5522050180086437
Decision Tree model best score with RMSE: 0.42651379771467124
Decision Tree model best params: {'max_depth': 5}


Random Forest model best score with RMSE: 0.5475364412116142
Random Forest model best score with RMSE: 0.4243204621404436
Random Forest model best score with RMSE: 0.6837700525896513
Random Forest model best params: {'n_estimators': 300}


XGBoost model best score with RMSE: 0.37387884000819566
XGBoost model best score with R

In [11]:
model_names = ['Linear Regression', 'Ridge', 'Decision Tree', 'Random Forest', 'XGBoost']
results = {'models': model_names,
           'Root Mean Square Error':rmse,
           'Mean Absolute Error': mae,
           'R2 score': r2}
# save the results into dataframe
models_scores_df = pd.DataFrame.from_dict(results)
models_scores_df

,models,Root Mean Square Error,Mean Absolute Error,R2 score
0,Linear Regression,0.535299,0.426242,0.697747
1,Ridge,0.534682,0.424424,0.698444
2,Decision Tree,0.737349,0.552205,0.426514
3,Random Forest,0.547536,0.424320,0.683770
4,XGBoost,0.373879,0.291598,0.852552


In [12]:
# XGBoot clasifier
# Load the model
model_xgb = XGBRegressor(objective='reg:squarederror', max_depth = 3, n_estimators = 600)

# Fitting the model with the training dataset
model_xgb.fit(X_processed, y_processed)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=600, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

### Part B

In [13]:
# HERE YOU WILL USE THIS TEMPLATE TO SAVE THE PREDICTIONS ON THE TEST SET

# Load the test data
test_df = pd.read_csv('CE802_P3_Test.csv')

# Make sure you work on a copy
test_data = test_df.iloc[:,:-1].copy()

test_data_processed = full_processor.fit_transform(test_data)
predicted = model_xgb.predict(test_data_processed)# CHANGE HERE -- use your previously trained predictor and apply it to test_data
                # (test_data can be modified if needed but make sure you don't change the order of the rows)... 
predicted = scaler_y.inverse_transform(predicted.reshape(-1, 1))
# Replace the last (empty) column with your prediction
test_df.iloc[:,-1] = predicted

# Save to the destination file
test_df.to_csv('CE802_P3_Test_Predictions.csv', index=False, float_format='%.8g')

# IMPORTANT!! Make sure only the last column has changed
assert pd.read_csv('CE802_P3_Test.csv').iloc[:,:-1].equals(pd.read_csv('CE802_P3_Test_Predictions.csv').iloc[:,:-1])